In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

from nlp_project.dataset import load_data
from nlp_project.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR, RAW_DATA_DIR
from nlp_project.plots import sb_bar_plot

In [ ]:
data_2018, metadata_2018 = load_data()

In [ ]:
print(data_2018.shape)
print(metadata_2018.shape)

In [ ]:
data_2018.info()

In [ ]:
metadata_2018.info()

In [ ]:
main_cat_value_counts = metadata_2018['main_cat'].value_counts()

In [ ]:
print(main_cat_value_counts)

In [ ]:
data_2018.head(20)

In [ ]:
metadata_2018.head()

In [ ]:
merged_df = pd.merge(data_2018, metadata_2018, on='asin')
print(merged_df.shape)
print(merged_df.info()) 

In [ ]:
# Select specific columns
selected_columns = ['overall', 'reviewText', 'category', 'description', 'title', 'brand', 'feature', 'details', 'main_cat', 'price', 'asin']
merged_df = merged_df[selected_columns]

In [ ]:
len(merged_df['asin'].unique())

In [ ]:
merged_df.head(30)

come ottengo solo le reviews di cellulari???
Andando a filtrare la categoria, selezionando solo ['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']

In [ ]:
categories_counts = merged_df['category'].value_counts()

In [ ]:
categories_counts = merged_df['category'].value_counts().reset_index()
categories_counts.columns = ['category', 'count']
#categories_counts
categories_counts.to_csv(INTERIM_DATA_DIR / 'full_categories_counts.csv', index = False) 

In [ ]:
#merged_df[merged_df['category'] == ['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']]
condition = merged_df['category'].isin([['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']])
print(merged_df['category'].isin([['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']]).value_counts())


In [ ]:
cell_phones_only = merged_df[merged_df['category'].isin([['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']])]
cell_phones_only

Quanti prodotti singoli ci sono?

In [ ]:
len(cell_phones_only['asin'].unique())

Quali sono i brand di cellulari più frequenti?

In [ ]:
cell_phones_only['brand'].value_counts()

In [ ]:
cell_phones_brand_counts = cell_phones_only['brand'].value_counts().reset_index()
cell_phones_brand_counts.columns = ['brand', 'count']
cell_phones_brand_counts.to_csv(INTERIM_DATA_DIR / 'brand_counts.csv', index = False) 

In [ ]:
#select top 10 brand
cell_phones_brand_counts = cell_phones_brand_counts[:10]
sb_bar_plot(x = cell_phones_brand_counts['count'],
            y = cell_phones_brand_counts['brand'],
            title = 'top 10 brand',
            xlabel = 'Count',
            ylabel = 'Brand',
            orient='h')

In [ ]:
top10_brand_list = cell_phones_brand_counts['brand']
cell_phones_top10_brand = cell_phones_only[cell_phones_only['brand'].isin(top10_brand_list)]
cell_phones_top10_brand

In [ ]:
cell_phones_top10_brand.isnull().sum()

In [ ]:
ratings = cell_phones_top10_brand['overall'].value_counts()
sb_bar_plot(x = ratings.values,
            y = ratings.index,
            title = 'Ratings value counts',
            xlabel = 'Count',
            ylabel = 'Rating',
            orient='h')

print(ratings)

In [ ]:
cell_phones_top10_brand = cell_phones_top10_brand.drop(columns = ['category', 'main_cat', 'details'])


In [ ]:
cell_phones_top10_brand.to_csv(PROCESSED_DATA_DIR / 'cell_phones_reviews.csv', index = False)